In [1]:
import os
import sys
from datetime import datetime
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 3 --executor-memory 3g --driver-memory 2g pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as t
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
import numpy as np
from numpy import dot
from numpy. linalg import norm
conf = SparkConf()

In [ ]:
spark = SparkSession\
    .builder\
    .config(conf=conf)\
    .appName("GF_spark_2")\
    .getOrCreate()

In [ ]:
spark

In [ ]:
sc = spark.sparkContext

In [ ]:
sc

### Load data

In [3]:
! hdfs dfs -ls /labs/slaba02/

Found 1 items
-rw-r--r--   3 hdfs hdfs   69519728 2022-01-06 18:46 /labs/slaba02/DO_record_per_line.json


In [ ]:
! hdfs dfs -copyToLocal /labs/slaba02/DO_record_per_line.json ~/

In [ ]:
df = spark.read\
     .format('json')\
     .load('/labs/slaba02/DO_record_per_line.json')
#df.show(1, vertical=True)

In [7]:
l = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'],
     [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'],
     [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'],
     [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'],
     [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'],
     [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]

### pre-map

In [8]:
df = df.withColumn('desc_arr', F.split(F.regexp_replace(F.lower(F.col('desc')), '[-—!,.:$&]', ''), ' '))\
    .select('id', 'lang', 'desc_arr')

In [9]:
def ff(x, y):
    for i in x:
        if i in y:
            return True
    return False

### post_map

In [17]:
def post(data, my_id):
    my_name_arr = df.filter(F.col('id') == my_id).collect()[0][2]
    my_lang = df.filter(F.col('id') == my_id).collect()[0][1]
    data = data.filter((F.col('lang') == my_lang)).select('id', 'desc_arr')
    rdd = data.rdd\
        .map(lambda x: [x[0], x[1]] + [ff(x[1], my_name_arr)])\
        .filter(lambda x: x[2] == True)\
        .map(lambda x: x[:-1])
    le = rdd.count()
    print(datetime.now().strftime("%H:%M:%S"))
    print(le)
    data = spark.createDataFrame(rdd)
    
    #TF IDF================================
    hashingTF = HashingTF(inputCol = '_2', outputCol = 'rawFeatures', numFeatures=10000)
    featurizedData = hashingTF.transform(data)
    idf = IDF(inputCol='rawFeatures', outputCol='features')
    idfModel = idf.fit(featurizedData)
    data = idfModel.transform(featurizedData).select(F.col('_1').alias('id'), F.col('features')) 
    
    #my vec cord=================
    my_vec = data.filter(F.col('id') == my_id).select('features').collect()[0]
    
    #cos calculation
    rdd = data.filter(F.col('id') != my_id).rdd\
        .map(lambda x: (x[0], dot(my_vec, x[1]) / (norm(my_vec) * norm(x[1]))))\
        .sortBy(lambda x: x[1], False)\
        
    
    
    return rdd

In [12]:
result = []
for i in l:
    result += post(df, i[0]).map(lambda x: x[0]).take(10)

09:50:00
24534
10:51:51
24532


KeyboardInterrupt: 

In [18]:
result = []
result += post(df, 16704).map(lambda x: x[0]).take(10)

12:52:22
1081


In [19]:
result

[1219, 1228, 1327, 20362, 26980, 1247, 913, 20095, 927, 20096]

In [20]:
[
        1247,
        913,
        20095,
        887,
        978,
        1332,
        20351,
        1250,
        1191,
        1219
    ]

[1247, 913, 20095, 887, 978, 1332, 20351, 1250, 1191, 1219]

### sample
{
  "123": [
    5372,
    16663,
    23114,
    13079,
    13084,
    ...
  ],
  "456": [
    ...
  ],
  "789": [
    ...
  ],
  "123456": [
    ...
  ],
  "456789": [
    ...
  ],
  "987654": [
    ...
  ]
}

### Finally!!!

In [16]:
d = {}
nn = 0
for i in range(6):
    d[str(l[i][0])] = result[nn : nn + 10]
    nn +=10

In [19]:
! touch lab02.json
file = open("lab02.json", "w")
file.write(str(d))
file.close()

In [17]:
d = 

{'23126': [15909, 14760, 24419, 13665, 4635, 25071, 26944, 4107, 13781, 12714],
 '21617': [21609,
  21623,
  21624,
  21628,
  21630,
  21608,
  21616,
  21700,
  21854,
  21492],
 '16627': [11431, 4188, 17964, 9598, 4093, 13551, 21704, 13552, 9041, 10142],
 '11556': [16488, 12679, 18005, 17910, 10781, 9985, 3872, 3874, 12978, 13180],
 '16704': [27383, 1247, 55, 1359, 20097, 1240, 768, 1011, 19924, 1229],
 '13702': [864, 853, 56, 1106, 13703, 7623, 16960, 20417, 787, 1109]}